## Speed dating data-set
#### Joris Rombouts - November 2017

### 1.3 Feature Engineering

### <font color="green">imports, preparation and configuration</font>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree, preprocessing, metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("speed_dating_assignment.csv")
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [3]:
#df['age_o'] = np.nan
df_o = df.copy()
df_o = df_o.filter(items=['iid', 'age', 'race', 'from', 'goal', 'attr3_2', 'attr5_2'])
df_o.rename(columns={'iid': 'pid', 'age': 'age_o', 'race': 'race_partner', 'from': 'from_partner', 'goal':'goal_partner'}, inplace = True)
df_o['attr5_2_diff'] = abs(df_o['attr3_2'] - df_o['attr5_2'])

df_o = df_o.drop_duplicates()
df_o = df_o.drop(['attr3_2', 'attr5_2'], axis = 1)

df_new = pd.merge(df, df_o, on=['pid'], how = 'left')

In [4]:
df =df_new.copy()

In [5]:
#df_new = df[(df['wave'] >= 6) & (df['wave'] <= 9)]
#df_new.shape

In [6]:
#df = pd.concat([df, df_new, df_new]).drop_duplicates(keep=False)
#df.head()

In [8]:
df = df[['attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'gender', 'order','goal', 'field_cd', 'date','age', 'go_out', 'imprace', 'imprelig', 'dec', 'age_o', 'from', 'race', 'pid', 'iid', 'race_partner', 'from_partner', 'goal_partner', 'attr5_2_diff']]
df.age.fillna(df.age.median(), inplace = True)
df.head()

C:\Users\Joris\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,attr,sinc,intel,fun,amb,shar,like,gender,order,goal,...,dec,age_o,from,race,pid,iid,race_partner,from_partner,goal_partner,attr5_2_diff
0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,0,4,2.0,...,1,27.0,Chicago,4.0,11,1,2.0,Argentina,1.0,NaN
1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,0,3,2.0,...,1,22.0,Chicago,4.0,12,1,2.0,England,1.0,NaN
2,5.0,8.0,9.0,8.0,5.0,7.0,7.0,0,10,2.0,...,1,22.0,Chicago,4.0,13,1,4.0,Southern California,2.0,NaN
3,7.0,6.0,8.0,7.0,6.0,8.0,7.0,0,5,2.0,...,1,23.0,Chicago,4.0,14,1,2.0,"Cincinnati, Ohio",2.0,NaN
4,5.0,6.0,7.0,7.0,6.0,6.0,6.0,0,7,2.0,...,1,24.0,Chicago,4.0,15,1,3.0,Miami,1.0,NaN


In [9]:
df = df.dropna().reset_index(drop=True)

In [10]:
df['attr_o'] = df['attr'] + df['sinc'] + df['intel'] + df['fun'] + df['shar'] + df['like'] + df['amb']
df = df.drop(['attr', 'sinc', 'intel', 'fun', 'shar', 'like', 'amb'], axis = 1)
df.head()

,gender,order,goal,field_cd,date,age,go_out,imprace,imprelig,dec,age_o,from,race,pid,iid,race_partner,from_partner,goal_partner,attr5_2_diff,attr_o
0,0,7,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,27.0,Virginia,6.0,243,234,2.0,"Portland, Oregon",2.0,0.0,43.0
1,0,4,3.0,3.0,7.0,22.0,3.0,1.0,5.0,1,28.0,Virginia,6.0,244,234,2.0,Seattle,2.0,1.0,51.0
2,0,6,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,32.0,Virginia,6.0,245,234,2.0,"Lexington, MA",4.0,0.0,45.0
3,0,2,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,25.0,Virginia,6.0,246,234,2.0,"brooklyn, ny",1.0,0.0,36.0
4,0,3,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,27.0,Virginia,6.0,248,234,6.0,San Francisco,1.0,1.0,39.0


In [11]:
df['age_diff'] = abs(df['age'] - df['age_o'])

In [12]:
df['race_equals'] = (df['race'] == df['race_partner'])
df['from_equals'] = (df['from'] == df['from_partner'])

In [13]:
df['goal_equals'] = df.apply(lambda r: r.goal == r.goal_partner, axis = 1)

In [14]:
df = df[['gender', 'order','goal', 'field_cd', 'date','age', 'go_out', 'imprace', 'imprelig', 'dec','age_diff', 'race_equals', 'from_equals', 'attr_o', 'goal_equals', 'attr5_2_diff']]
df.head()

,gender,order,goal,field_cd,date,age,go_out,imprace,imprelig,dec,age_diff,race_equals,from_equals,attr_o,goal_equals,attr5_2_diff
0,0,7,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,5.0,False,False,43.0,False,0.0
1,0,4,3.0,3.0,7.0,22.0,3.0,1.0,5.0,1,6.0,False,False,51.0,False,1.0
2,0,6,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,10.0,False,False,45.0,False,0.0
3,0,2,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,3.0,False,False,36.0,False,0.0
4,0,3,3.0,3.0,7.0,22.0,3.0,1.0,5.0,0,5.0,True,False,39.0,False,1.0


### train models

In [15]:
male_subjects = df[df.gender == 1]
female_subjects = df[df.gender == 0]
male_subjects.shape, female_subjects.shape

((1784, 16), (1789, 16))

In [16]:
male_subjects_shuffle = male_subjects.sample(frac=1).reset_index(drop=True) # shuffle rows
female_subjects_shuffle = female_subjects.sample(frac=1).reset_index(drop=True)

In [17]:
male_x_data = male_subjects_shuffle.drop('dec', axis = 1) # dec is target attribute
female_x_data = female_subjects_shuffle.drop('dec', axis = 1)
male_labels = male_subjects_shuffle['dec']
female_labels = female_subjects_shuffle['dec']

In [18]:
male_x_train, male_x_test, male_y_train, male_y_test = train_test_split(male_x_data, male_labels, test_size = 0.2)
female_x_train, female_x_test, female_y_train, female_y_test = train_test_split(female_x_data, female_labels, test_size = 0.2)

In [19]:
#np.all(np.isfinite(df))

In [20]:
#np.any(np.isnan(df))

In [21]:
dec_tree_male = tree.DecisionTreeClassifier(criterion = "gini", max_depth = 8, min_samples_split = 50, min_samples_leaf = 12)
dec_tree_female = tree.DecisionTreeClassifier(criterion = "gini", max_depth = 8, min_samples_split = 50, min_samples_leaf = 12)
# build decision tree classifiers from the training sets
dec_tree_male.fit(male_x_train, male_y_train)
dec_tree_female.fit(female_x_train, female_y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=12, min_samples_split=50,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [22]:
pred_male = dec_tree_male.predict(male_x_test) # predicting 'dec' for data which the model has not seen yet
print("Accuracy score: ", metrics.accuracy_score(male_y_test, pred_male))

Accuracy score:  0.717086834734


In [23]:
pred_female = dec_tree_female.predict(female_x_test) # predicting 'dec' for data which the model has not seen yet
print("Accuracy score: ", metrics.accuracy_score(female_y_test, pred_female))

Accuracy score:  0.751396648045


In [24]:
print(metrics.classification_report(male_y_test, pred_male))

             precision    recall  f1-score   support

          0       0.72      0.72      0.72       179
          1       0.72      0.72      0.72       178

avg / total       0.72      0.72      0.72       357



In [25]:
print(metrics.classification_report(female_y_test, pred_female))

             precision    recall  f1-score   support

          0       0.77      0.85      0.81       219
          1       0.72      0.59      0.65       139

avg / total       0.75      0.75      0.75       358

